# Vehicular Coordination for lane change prediction
    AIM - Develop a driver assist system to aid in lane change and overtaking decision making.
 
    OBJECTIVE - To minimise the rate of collissions related to overtaking and lane change maneuvres.

In [ ]:
# Required libraries
import scipy
import numpy as np
import pandas as pd
import sklearn
import statsmodels
import tensorflow as tf
import os
import platform

In [ ]:
# Check the version of the above libraries currently installed
print(f"Scipy Version: {scipy.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Sklearn Version: {sklearn.__version__}")
print(f"statsmodels Version: {statsmodels.__version__}")
print(f"Tensorflow Version: {tf.__version__}")
version = platform.python_version()
print("Python : ",version)

# Data Visualization and Exploration

##### A brief overview and details pertaining the dataset can be found from the official website
<a href = "https://www.highd-dataset.com/format" >The official HighD Dataset </a>

The data consists of 6o recordings, with each recording containing 4 files.
<blockquote>
    <ol>
        <li>Image of the recorded highway section</li>
        <li>csv file detailing the recorded location</li>
        <li>csv file containing an overview of the recorded vehicle tracks</li>
        <li>csv file for the tracks</li>
    </ol>

  </blockquote>
  
  ### Sample Recorded Highway Section

![](data/01_highway.png)

## Location Metadata
Contains metadata for each recording.
<blockquote>
    <ol>
        <li>Time of recording</li>
        <li>Highway Section Considered</li>
        <li>Total number of vehicles recorded.</li>
    </ol>
    </blockquote>
    
        

In [8]:
# Overview of the file
recordingMeta = pd.read_csv("data/01_recordingmeta.csv")
recordingMeta.head()

,id,frameRate,locationId,speedLimit,month,weekDay,startTime,duration,totalDrivenDistance,totalDrivenTime,numVehicles,numCars,numTrucks,upperLaneMarkings,lowerLaneMarkings
0,1,25,2,-1.0,9.2017,Tue,08:38,901.56,418549.19,13908.12,1047,863,184,8.51;12.59;16.43,21.00;24.96;28.80


In [18]:
columns = recordingMeta.columns
description = ["The id of the recording. Every recording has a unique id.",
               "The frame rate which was used to record the video.",
               "The id of the recording location. In total six different locations exist in the dataset.",
               "The speed limit of the driving lanes. In all recordings, the speed limit is the same for every driving lane.",
              "The month the recording was done. ",
              "The week day the recording was done. . ",
              "The start time at which the recording was done. ",
              "The duration of the recording. ",
              "The total driven distance of all tracked vehicles. ",
              "The total driven time of all tracked vehicles. ",
              "The number of vehicles tracked including cars and trucks. ",
              "The number of cars tracked. ",
              "The number of trucks tracked. ",
              "The y positions of the upper lane markings. The positions are separated by a ';'.",
              "The y positions of the lower lane markings. The positions are separated by a ';'.",]

for i in range(len(columns)):
    print(f'{columns[i]} : {description[i]} > {recordingMeta[columns[i]][0]}')
    print()

id : The id of the recording. Every recording has a unique id. > 1

frameRate : The frame rate which was used to record the video. > 25

locationId : The id of the recording location. In total six different locations exist in the dataset. > 2

speedLimit : The speed limit of the driving lanes. In all recordings, the speed limit is the same for every driving lane. > -1.0

month : The month the recording was done.  > 9.2017

weekDay : The week day the recording was done. .  > Tue

startTime : The start time at which the recording was done.  > 08:38

duration : The duration of the recording.  > 901.56

totalDrivenDistance : The total driven distance of all tracked vehicles.  > 418549.19

totalDrivenTime : The total driven time of all tracked vehicles.  > 13908.12

numVehicles : The number of vehicles tracked including cars and trucks.  > 1047

numCars : The number of cars tracked.  > 863

numTrucks : The number of trucks tracked.  > 184

upperLaneMarkings : The y positions of the upper la

### Track Meta Information
Contains metadata for each Track.
<blockquote>
    <ol>
        <li>Distance Covered</li>
        <li>Average Speed</li>
    </ol>
    </blockquote>
<strong>This file is purposed to help filter tracks by class or driving direction</strong>

In [23]:
# Overview of the file
# Overview of the file
tracksMeta = pd.read_csv("data/01_tracksMeta.csv")
tracksMeta.head()

,id,width,height,initialFrame,finalFrame,numFrames,class,drivingDirection,traveledDistance,minXVelocity,maxXVelocity,meanXVelocity,minDHW,minTHW,minTTC,numLaneChanges
0,1,4.85,2.12,1,33,33,Car,2,52.25,40.85,41.30,41.07,-1.00,-1.00,-1.00,0
1,2,4.24,1.92,1,130,130,Car,1,167.44,32.04,32.90,32.48,112.62,3.51,-1.00,0
2,3,3.94,1.92,1,157,157,Car,2,225.23,35.69,36.50,36.13,90.31,2.53,-1.00,0
3,4,5.05,2.22,1,161,161,Car,1,273.49,42.57,42.83,42.76,-1.00,-1.00,-1.00,0
4,5,4.24,1.82,1,182,182,Car,1,313.92,42.50,44.26,43.40,33.64,0.76,24.53,0


#### Column Description

In [41]:
tracks_cols = tracksMeta.columns
tracks_desc = ["The id of the track. The ids are assigned in ascending order. ",
              "The width of the post-processed bounding box of the vehicle. This corresponds to the length of the vehicle. ",
              "The height of the post-processed bounding box of the vehicle. This corresponds to the height of the vehicle. ",
              "The initial frame in which the vehicle track starts. ",
              "The frame in which the track of the vehicle ends. ",
              "The total lifetime of the track as number of frames. ",
              "The vehicle class of the tracked vehicle (Car or Truck).",
              "The driving direction of the vehicle. Either 1 for the left direction (upper lanes) or 2 for the right direction (lower lanes). ",
              "The distance covered by the track.",
              "The minimal velocity in driving direction. ",
              "The maximal velocity in driving direction.",
              "The mean velocity in driving direction. ",
              " The minimal Distance Headway (DHW). This value is set to -1, if no preceding vehicle exists. ",
              "The minimal Time Headway (THW). This value is set to -1, if no preceding vehicle exists. ",
              "The minimal Time-to-Collision (TTC). This value is set to -1, if no preceding vehicle or valid TTC exists. ",
              "Number of lane changes detected by changing lane id",
              ]

for i in range(len(tracks_cols)):
    print(f'{tracks_cols[i]} : {tracks_desc[i]} > {tracksMeta[tracks_cols[i]][0]}')
    print()

id : The id of the track. The ids are assigned in ascending order.  > 1

width : The width of the post-processed bounding box of the vehicle. This corresponds to the length of the vehicle.  > 4.85

height : The height of the post-processed bounding box of the vehicle. This corresponds to the height of the vehicle.  > 2.12

initialFrame : The initial frame in which the vehicle track starts.  > 1

finalFrame : The frame in which the track of the vehicle ends.  > 33

numFrames : The total lifetime of the track as number of frames.  > 33

class : The vehicle class of the tracked vehicle (Car or Truck). > Car

drivingDirection : The driving direction of the vehicle. Either 1 for the left direction (upper lanes) or 2 for the right direction (lower lanes).  > 2

traveledDistance : The distance covered by the track. > 52.25

minXVelocity : The minimal velocity in driving direction.  > 40.85

maxXVelocity : The maximal velocity in driving direction. > 41.3

meanXVelocity : The mean velocity in 

In [28]:
tracksMeta.describe()

,id,width,height,initialFrame,finalFrame,numFrames,drivingDirection,traveledDistance,minXVelocity,maxXVelocity,meanXVelocity,minDHW,minTHW,minTTC,numLaneChanges
count,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000
mean,524.000000,6.249914,2.086982,10929.094556,11261.189112,333.094556,1.432665,399.760487,30.132798,32.223009,31.170287,63.569312,2.056925,150.560669,0.132760
std,302.387169,3.878263,0.251938,6487.928248,6487.789075,67.821127,0.495682,32.916233,5.551943,5.914268,5.706401,63.201659,2.093757,2240.063766,0.353278
min,1.000000,2.630000,1.720000,1.000000,29.000000,28.000000,1.000000,41.540000,20.130000,21.400000,20.900000,-1.000000,-1.000000,-1.000000,0.000000
25%,262.500000,4.350000,1.920000,5380.000000,5781.500000,289.000000,1.000000,403.220000,24.805000,27.360000,26.165000,23.375000,0.780000,-1.000000,0.000000
50%,524.000000,4.750000,2.020000,10480.000000,10784.000000,320.000000,1.000000,405.560000,30.320000,32.600000,31.610000,40.330000,1.300000,17.050000,0.000000
75%,785.500000,5.150000,2.220000,16397.500000,16720.500000,384.500000,2.000000,406.935000,33.865000,36.105000,34.935000,79.950000,2.580000,43.465000,0.000000
max,1047.000000,19.400000,3.230000,22508.000000,22539.000000,486.000000,2.000000,411.960000,48.380000,50.260000,49.480000,388.730000,13.330000,70279.150000,2.000000


From the above description, we can quickly get a glimpse of the overall dataset.
It is clear that there are 1047 total tracks.

In [34]:
tracksMeta.drivingDirection.value_counts()

1    594
2    453
Name: drivingDirection, dtype: int64

In [40]:
tracksMeta["class"].value_counts()

Car      863
Truck    184
Name: class, dtype: int64

We can quickly deduce that the dataset has vehicles driving in both direction.
The dataset also has a considerable number of cars to trucks. Which is also visible uder the data descriptions width mean.

### Track Information
Contains all time dependent values for each track.
<blockquote>
    <ol>
        <li>Current velocities, </li>
        <li>Viewing ranges </li>
        <li>Information about surrounding vehicles</li>
    </ol>
    </blockquote>



In [27]:
# Overview of the file
tracks = pd.read_csv("data/01_tracks.csv")
tracks.head()

,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
0,1,1,362.26,21.68,4.85,2.12,40.85,0.00,0.30,0.0,...,0.0,0,3,0,0,0,0,0,6,5
1,2,1,363.73,21.68,4.85,2.12,40.87,0.00,0.30,0.0,...,0.0,0,14,0,0,0,0,0,6,5
2,3,1,365.27,21.68,4.85,2.12,40.88,0.00,0.31,0.0,...,0.0,0,14,0,0,0,0,0,6,5
3,4,1,366.83,21.68,4.85,2.12,40.89,0.00,0.32,0.0,...,0.0,0,14,0,0,0,0,0,6,5
4,5,1,368.42,21.68,4.85,2.12,40.90,0.01,0.32,0.0,...,0.0,0,14,0,0,0,0,0,6,5


#### Columns Description

In [43]:
# Overview of the file
tracks_cols = tracks.columns
tracks_desc = ["The current frame. ",
              "The track's id. ",
              "The x position of the upper left corner of the vehicle's bounding box. ",
              "The y position of the upper left corner of the vehicle's bounding box. 	",
              "The width of the bounding box of the vehicle. ",
              "The height of the bounding box of the vehicle. ",
              "The longitudinal velocity in the image coordinate system. ",
              "The lateral velocity in the image coordinate system. ",
              "The longitudinal acceleration in the image coordinate system.",
              "The lateral acceleration in the image coordinate system ",
              "The distance to the end of the recorded highway section in driving direction from the vehicle's center. ",
              "The distance to the end of the recorded highway section in the opposite driving direction from the vehicle's center. ",
              "The Distance Headway. This value is set to 0, if no preceding vehicle exists. ",
              "The Time Headway. This value is set to 0, if no preceding vehicle exists. ",
              "The Time-to-Collision. This value is set to 0, if no preceding vehicle or valid TTC exists. ",
              "The longitudinal velocity of the preceding in the image coordinate system. This value is set to 0, if no preceding vehicle exists.",
              "The id of the preceding vehicle in the same lane. This value is set to 0, if no preceding vehicle exists.",
              "The id of the following vehicle in the same lane. This value is set to 0, if no following vehicle exists.",
              "The id of the preceding vehicle on the adjacent lane on the left in the direction of travel. This value is set to 0, if no such a vehicle exists. ",
              "The id of the adjacent vehicle on the adjacent lane on the left in the direction of travel. In order for a vehicle to be adjacent and not e.g. preceding, the vehicles must overlap in the longitudinal direction. This value is set to 0, if no such a vehicle exists. ",
              "The id of the following vehicle on the adjacent lane on the left in the direction of travel. This value is set to 0, if no such a vehicle exists. ",
              "The id of the preceding vehicle on the adjacent lane on the right in the direction of travel. This value is set to 0, if no such a vehicle exists.",
              "The id of the adjacent vehicle on the adjacent lane on the right in the direction of travel. In order for a vehicle to be adjacent and not e.g. preceding, the vehicles must overlap in the longitudinal direction. This value is set to 0, if no such a vehicle exists. ",
              "The id of the following vehicle on the adjacent lane on the right in the direction of travel. This value is set to 0, if no such a vehicle exists. ",
              " The IDs start at 1 and are assigned in ascending order. Since the Lane ids are derived from the positions of the lane markings, the first and last ids typically do not describe any useable lanes. For details, see the definition of the coordinate system. ",
              ]

In [29]:
tracks.describe()

,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
count,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,...,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000,348750.000000
mean,11092.433683,523.745746,204.281104,16.661879,6.717039,2.113467,-4.010287,-0.001647,0.054728,0.007636,...,-3.869772,391.717230,394.146724,181.105222,44.875352,196.544981,219.968118,44.810486,210.516416,3.795289
std,6375.582063,297.030503,117.204488,6.404551,4.272276,0.264999,30.378732,0.180925,0.296237,0.082029,...,26.096451,340.256519,341.188604,307.114828,168.675132,315.115893,316.918594,168.252081,314.937564,1.551607
min,1.000000,1.000000,-15.680000,8.200000,2.630000,1.720000,-48.870000,-1.790000,-2.950000,-0.750000,...,-48.870000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
25%,5761.000000,269.000000,102.940000,9.870000,4.350000,1.920000,-30.920000,-0.060000,-0.050000,-0.030000,...,-29.030000,7.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,10529.000000,517.000000,204.115000,13.500000,4.750000,2.020000,-23.540000,0.000000,0.040000,0.010000,...,0.000000,343.000000,347.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,16451.000000,780.000000,305.470000,22.250000,6.060000,2.430000,29.080000,0.060000,0.160000,0.050000,...,23.410000,688.000000,692.000000,287.000000,0.000000,350.000000,411.000000,0.000000,385.000000,5.000000
max,22539.000000,1047.000000,414.520000,27.490000,19.400000,3.230000,50.260000,1.750000,2.230000,0.900000,...,50.260000,1045.000000,1047.000000,1045.000000,1045.000000,1047.000000,1046.000000,1046.000000,1046.000000,6.000000


    It is clear that the tacks are distinct as the previous file.
    It can also be deduced that this file is a high level summary of the vehicles interaction within the environment under study.
    

In [44]:
for i in range(len(tracks_cols)):
    print(f'{tracks_cols[i]} : {tracks_desc[i]} > {tracks[tracks_cols[i]][0]}')
    print()
    

frame : The current frame.  > 1

id : The track's id.  > 1

x : The x position of the upper left corner of the vehicle's bounding box.  > 362.26

y : The y position of the upper left corner of the vehicle's bounding box. 	 > 21.68

width : The width of the bounding box of the vehicle.  > 4.85

height : The height of the bounding box of the vehicle.  > 2.12

xVelocity : The longitudinal velocity in the image coordinate system.  > 40.85

yVelocity : The lateral velocity in the image coordinate system.  > 0.0

xAcceleration : The longitudinal acceleration in the image coordinate system. > 0.3

yAcceleration : The lateral acceleration in the image coordinate system  > 0.0

frontSightDistance : The distance to the end of the recorded highway section in driving direction from the vehicle's center.  > 48.05

backSightDistance : The distance to the end of the recorded highway section in the opposite driving direction from the vehicle's center.  > 363.77

dhw : The Distance Headway. This value 

### Coordinate System
    The global coordinate system used corresponds to the image coordinate system of the video recording.
    Origin of the coordinate system is in the upper left corner
    The horizontal axis is the x-axis, and grows to the right
    The vertical axis is the y-axis. However, this axis grows downwards.
    